A model to predict the next word
Approach: A wholesome vocabulary has been made, and encoded into integers

Input Array contains: 1st, 2nd, 3rd word column and next(4th word in sequence)

## Remove punctuation marks from words

In [ ]:
import re

def remove_spl_char(tokenized_sent):
    
    punctuation = re.compile('[^a-zA-Z]*')
    post_punct_tokenized_sent = []
    for token in tokenized_sent:
        word=punctuation.sub("", token.lower())
        if len(word) > 0:
            post_punct_tokenized_sent.append(word)

    return post_punct_tokenized_sent

## Read corpus files

In [ ]:
from nltk.corpus import gutenberg
all_tokenized_sentences = []
for name in gutenberg.fileids():
    tokenized_sentences = gutenberg.sents(name)
    all_tokenized_sentences.extend(tokenized_sentences)

In [ ]:
all_tokenized_sentences=all_tokenized_sentences[:200]

## Process sentence tokens

In [ ]:
processed_sentences = [remove_spl_char(tokenized_sent) for tokenized_sent in all_tokenized_sentences]
processed_sentences = [sent for sent in processed_sentences if len(sent) > 3] 

# Enumerate Vocab

In [ ]:
vocab = list(set([word for sent in processed_sentences for word in sent]))
len(vocab)

In [ ]:
dictionary = {b : a for (a, b) in enumerate(vocab)}
reverse_dictionary = {a : b for (a, b) in enumerate(vocab)}

## Prepare input and target arrays

In [ ]:
import numpy as np
from nltk.util import ngrams
trigrams = ngrams
word_input = np.array([words for sent in processed_sentences for words in ngrams(sent[:-1], 3)])
word_output = np.array([words for sent in processed_sentences for words in sent[3:]])

In [ ]:
from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()
word_output_encoded=label.fit_transform(word_output)

label_classes = to_categorical(word_output_encoded.reshape(-1, 1))

In [ ]:
import pandas as pd
data = pd.DataFrame({"Input1" : [dictionary[x] for x in word_input[:,0]], 
                     "Input2" : [dictionary[x] for x in word_input[:,1]], 
                     "Input3" : [dictionary[x] for x in word_input[:,2]], 
                     "Output" : label_class}).to_numpy()
data=data.reshape(len(data), 4, 1)
data

## Import modules

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from sklearn.model_selection import train_test_split
import numpy as np
from keras.losses import categorical_crossentropy

## Split Train and Test Data

In [ ]:
train_size = int(len(data)*0.8)

## Construct LSTM

In [ ]:
model = Sequential()
model.add(LSTM((500), activation = 'relu', batch_input_shape=(None, 3, 1), return_sequences = True))
model.add(LSTM((984), activation = 'softmax', return_sequences = False))
model.compile(loss=categorical_crossentropy, optimizer='sgd', metrics=['accuracy'])

model.summary()

## Train with data

In [ ]:
history = model.fit(data[:train_size, 0:3], 
                    label_classes[:train_size], epochs = 5, 
                    validation_data = (data[train_size: , 0:3], label_classes[train_size:]))

## Analysis

In [ ]:
def get_word(num):
    return reverse_dictionary[num]

p=model.predict_classes(data[train_size: , 0:3])